In [2]:
import json

In [53]:
with open('../datasets/coco/annotations/instances_train2017.json', 'rt', encoding='UTF-8') as annotations:
    check = json.load(annotations)

In [17]:
images = check['images']

In [59]:
annotations = check['annotations']

In [62]:
check['licenses']

[{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/',
  'id': 1,
  'name': 'Attribution-NonCommercial-ShareAlike License'},
 {'url': 'http://creativecommons.org/licenses/by-nc/2.0/',
  'id': 2,
  'name': 'Attribution-NonCommercial License'},
 {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/',
  'id': 3,
  'name': 'Attribution-NonCommercial-NoDerivs License'},
 {'url': 'http://creativecommons.org/licenses/by/2.0/',
  'id': 4,
  'name': 'Attribution License'},
 {'url': 'http://creativecommons.org/licenses/by-sa/2.0/',
  'id': 5,
  'name': 'Attribution-ShareAlike License'},
 {'url': 'http://creativecommons.org/licenses/by-nd/2.0/',
  'id': 6,
  'name': 'Attribution-NoDerivs License'},
 {'url': 'http://flickr.com/commons/usage/',
  'id': 7,
  'name': 'No known copyright restrictions'},
 {'url': 'http://www.usa.gov/copyright.shtml',
  'id': 8,
  'name': 'United States Government Work'}]

In [61]:
check['categories']

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [6]:
images[0]

{'license': 3,
 'file_name': '000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 391895}

In [7]:
annotations[0]

{'segmentation': [[239.97,
   260.24,
   222.04,
   270.49,
   199.84,
   253.41,
   213.5,
   227.79,
   259.62,
   200.46,
   274.13,
   202.17,
   277.55,
   210.71,
   249.37,
   253.41,
   237.41,
   264.51,
   242.54,
   261.95,
   228.87,
   271.34]],
 'area': 2765.1486500000005,
 'iscrowd': 0,
 'image_id': 558840,
 'bbox': [199.84, 200.46, 77.71, 70.88],
 'category_id': 58,
 'id': 156}

In [8]:
len(images)

118287

In [9]:
import funcy

In [19]:
image_with_annotations = funcy.lmap(lambda a: a['image_id'], annotations)

In [ ]:
images = funcy.lfilter(lambda a: a['id'] in image_with_annotations, images[:1000])

In [16]:
len(images)

99

In [ ]:
image_id = 0
json_dict = {"images": [],  "annotations": [], "categories": [{"id": 1, "name": "element"}]}

for brand in os.listdir('detectron2-1/data/Database_Alexa_train'):
    filename = brand + '/shot.png'
    coord_path = os.path.join('detectron2-1/data/Database_Alexa_train', brand, 'final_coordinates.txt')

    img = cv2.imread(os.path.join('detectron2-1/data/Database_Alexa_train', filename))
    img_height, img_width, _ = img.shape
    
    image = {
        "file_name": filename,
        "height": img_height,
        "width": img_width,
        "id": image_id,
    }

    json_dict["images"].append(image)
    
    
    ## get gt_box annotations
    ele_names = [x.strip().split('\t')[0] for x in open(coord_path).readlines()]
    coords = [x.strip().split('\t')[1] for x in open(coord_path).readlines()]

    for i, b in enumerate(coords):
        x1, y1, x2, y2 = list(map(int, re.search(r'\((.*?)\)', b).group(1).split(",")))
        width = x2 - x1
        height = y2 - y1

        category_id = 1
        id_annot = json_dict['annotations'][-1]['id']+1 if len(json_dict["annotations"])!=0 else 0

        ann = {
            "area": width * height,
            "image_id": image_id,
            "bbox": [x1, y1, width, height],
            "category_id": category_id,
            "id": id_annot, # id for box, need to be continuous
            "iscrowd": 0
            }
        
        json_dict["annotations"].append(ann)
        
    image_id += 1
    ## write to json file
    with open('detectron2-1/data/layout_train_complete.json', "w") as f:
        json.dump(json_dict, f)

In [66]:
import cv2
import os
import re

In [76]:
class CreateCocoJson():
    
    def __init__(self, imgpath, jsonpath, annot=True, annot_path=None):
        
        self.imgpath = imgpath
        self.jsonpath = jsonpath
        self.annot = annot
        self.annot_path = annot_path
        if annot:
            self.datadict = {'images':[], 'annotations':[], "categories": check['categories']}
        else:
            self.datadict = {'images':[], "categories": check['categories']}
    
    def _write_images(self):
        
        image_id = 0
        for file in os.listdir(self.imgpath):
            filename = os.path.join(self.imgpath, file)
#             print(filename)
            img = cv2.imread(filename)
            img_height, img_width, _ = img.shape

            image = {
                "file_name": file,
                "height": img_height,
                "width": img_width,
                "id": image_id,
            }

            self.datadict["images"].append(image)
            if self.annot:
                self._write_annotations(file, image_id)

            image_id += 1
#             if image_id % 100 == 0:
#                 print(self.datadict)

        
    def _write_annotations(self, filename, image_id):
        
        annotations = open(self.annot_path).readlines()
        filenames = [x.strip().split('\t')[0] for x in annotations]
        coords = [x.strip().split('\t')[1] for x in annotations]
        cls = [x.strip().split('\t')[2] for x in annotations]

        for k, file in enumerate(filenames):
            if file == filename:
                x1, y1, x2, y2 = list(map(int, re.search(r'\((.*?)\)', coords[k]).group(1).split(",")))
                width = max(0, x2 - x1)
                height = max(0, y2 - y1)

                category_id = cls[k]
                id_annot = self.datadict['annotations'][-1]['id']+1 if len(self.datadict["annotations"])!=0 else 0

                ann = {
                    "area": width * height,
                    "image_id": image_id,
                    "bbox": [x1, y1, width, height],
                    "category_id": category_id,
                    "id": id_annot, # id for box, need to be continuous
                    "iscrowd": 0
                    }

                self.datadict["annotations"].append(ann)
                    
    
    def main(self):
        self._write_images()
        with open(self.jsonpath, 'wt', encoding='UTF-8') as f:
            json.dump(self.datadict, f)


In [77]:
jsonwriter = CreateCocoJson(imgpath='../datasets/coco/unlabeled2017/unlabeled2017/', 
                            jsonpath='../datasets/coco/annotations/coco_2017_unlabel.json',
                            annot=False)

In [78]:
jsonwriter.main()